# 1. 텍스트 수집

- 영문 & 한글 데이터

### (참고) text data download

#### 영어)
- https://www.kaggle.com/datsets
- https://archive.ics.uci.edu/ml/index.php
- http://ana.cachopo.org/datasets-for-single-label-text-categorization
- https://course.fast.ai/datasets

#### 한글)
- https://korquad.github.io/
- https://ithub.korean.go.kr/user/guide/corpus/guide1.do
- https://konlpy-ko.readthedocs.io/ko/v0.5.1/data;
- http://aiopen.etri.re.kr/service_dataset.php

## [1] URL 가져오기

In [1]:
import os
import sys
import urllib.request
import requests

In [2]:
client_id = "D4FywX610oOr4AkcwJBk"
client_secret = "Pdhm4LS7nP"

- '방탄소년단'이 들어간 page 5장

In [8]:
encText = urllib.parse.quote("방탄소년단")
news_data = []
page_cnt = 5


for idx in range(page_cnt):
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + '&start=' + str(idx*10+1)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode==200:
        result = requests.get(response.geturl(),
                             headers={'X-Naver-Client-Id':client_id,
                                     'X-Naver-Client-Secret':client_secret})
        news_data.append(result.json())
    else:
        print("Error Code:" + rescode)

In [9]:
naver_news_link = []

for page in news_data :
    
    page_news_link = []    

    for item in page['items']:
        link = item['link']
        if 'naver' in link :
            page_news_link.append(link)
            
    naver_news_link.append(page_news_link)

In [17]:
print('number of pages :', len(naver_news_link))
print('links per pages :', len(naver_news_link[0]),
      len(naver_news_link[1]),
      len(naver_news_link[2]),
      len(naver_news_link[3]),
      len(naver_news_link[4]))
      

number of pages : 5
links per pages : 6 8 4 3 8


In [15]:
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=382&aid=0000801216
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=112&aid=0003272117
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=311&aid=0001116943
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=112&aid=0003272099
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=609&aid=0000252147
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=008&aid=0004364449
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=025&aid=0002978803
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640658
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=312&aid=0000437320
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640619
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640620
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&

## [2] Selenium으로 crawling

In [11]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import time

In [14]:
driver = webdriver.Chrome('driver/chromedriver')

In [18]:
naver_news_title = []
naver_news_content = []

for n in tqdm_notebook(range(len(naver_news_link))):
    # per page
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        ############### (1) URL로 접속하기 ###############
        try :
            link = naver_news_link[n][idx]
            driver.get(link)
            print(link)
        except :
            print('Timeout!')
            continue
        
        try :
            response = driver.page_source        
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print('게시글이 삭제된 경우입니다')
            continue
        
        ############## (2) news 제목 가져오기 #############
        soup = BeautifulSoup(response, 'html.parser')
        title = None
        
        try :
            item = soup.find('div', class_ = 'article_info')
            title = item.find('h3', class_ = 'tts_head').get_text()
        except:
            title = 'OUTLINK'
        
        news_page_title.append(title)
            
        ############## (3) news 본문 가져오기 #############
        doc = None
        text = ""
        
        data = soup.find_all('div', {'class' : '_article_body_contents'})
        if data :
            for item in data:
                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
        else:
            doc = 'OUTLINK'
        
        news_page_content.append(doc.replace('\n',' '))
    
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)
    
    time.sleep(2)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=382&aid=0000801216
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=112&aid=0003272117
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=311&aid=0001116943
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=112&aid=0003272099
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=609&aid=0000252147
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=008&aid=0004364449


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=025&aid=0002978803
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640658
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=312&aid=0000437320
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640619
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0001640620
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=079&aid=0003327947
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=016&aid=0001640598
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=021&aid=0002418655


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=011&aid=0003700540
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=088&aid=0000635100
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002845626
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=052&aid=0001406199


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=144&aid=0000658402
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=410&aid=0000672338
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=468&aid=0000630695


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=311&aid=0001116870
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=057&aid=0001425190
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=408&aid=0000087546
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=374&aid=0000205821
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=629&aid=0000016847
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=427&aid=0000035731
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=427&aid=0000035730
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=427&aid=0000035729



In [19]:
print(naver_news_title[0])
print("-------------------------")
print(naver_news_content[0])

['OUTLINK', 'OUTLINK', 'OUTLINK', 'OUTLINK', 'OUTLINK', 'BTS, 일본 1위 찍고 미국·영국 ‘정상’ 돌진']
-------------------------
['OUTLINK', 'OUTLINK', 'OUTLINK', 'OUTLINK', 'OUTLINK', "   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}   \t \t[머니투데이 김고금평 기자] [24일자 오리콘 데일리 앨범 차트 1위…22만 7204장 “미국과 영국 석권할 것”] 방탄소년단. /사진제공=빅히트엔터테인먼트 BTS(방탄소년단)가 일본 1위를 찍고 미국과 영국 차트 1위에 도전한다. BTS가 최근 내놓은 신보 ‘맵 오브 더 소울’(MAP OF THE SOUL : 7)이 24일 일본 오리콘 데일리 앨범 차트 정상에 올랐다. 이날 일본 오리콘이 발표한 랭킹 차트에 따르면 지난 21일 전 세계에서 발매한 BTS의 정규 4집 ‘MAP OF THE SOUL : 7’은 총 22만 7204장의 판매고로 24일자 오리콘 데일리 앨범 차트 1위를 차지했다. ‘MAP OF THE SOUL : 7’은 국내 주요 음원사이트의 실시간 차트 1위를 비롯한 차트 줄세우기, 공개 6시간 만에 중국 왕이뮤직 디지털 앨범 판매량 33만 장 돌파, 전 세계 91개 국가 및 지역 아이튠즈 ‘톱 앨범’ 1위, 발매 당일 265만 장 이상 판매 등 컴백 첫날부터 순조로운 기록들을 써내려가고 있다. 또 지난 24일 미국 빌보드와 영국 오피셜 차트는 이번 앨범이 전작의 성과를 뛰어넘어 ‘빌보드 200’ 차트와 ‘오피셜 앨범차트 톱 100’ 차트를 동시 석권할 것이라는 전망의 기사를 보도하며 BTS의 신기록을 집중적으로 다뤘다. 새 음반은 한 팀으로 모인 7명이 데뷔 7년을 돌아보는 작품이다. 세상에 ‘보여주고 싶은 나’와 그동안 숨겨왔던 내면의 그림자, ‘외면하고 

# Saving

In [21]:
with open('naver_news_title.pk','wb') as f:
    pickle.dump(naver_news_title,f)

with open('naver_news_content.pk','wb') as f:
    pickle.dump(naver_news_title, f)